<a href="https://colab.research.google.com/github/ducthuy-ng/computer_vision_sem_241/blob/main/DLACV_Ass1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [1]:
# Import libraries
import torch
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from torchvision.models import resnet18

In [2]:
# Set device
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

## Custom Model for CIFAR-10 classification

1. **(3pts)** Create a Resnet-18 or RegNet-Y or MobileNet for CIFAR-10 classification

- (a) Train model from scratch without using pretrained weights from ImageNet; evaluate.

- (b) Train model using pretrained weights from ImageNet (fine-tune transfer learning); evaluate. The performance must higher than results from (a). Provide discussion for the results.

2. **(4pts)** Repeat requirements from (1-a) and (1-b) but add data augmentation with methods from https://github.com/albumentations-team/albumentations . The performance must higher than results from (1) Provide discussion for the results.

3. **(3pts)** Find best learning rate for models from (1) and (2) with learning rate finder from https://github.com/surmenok/keras_lr_finder or other similar approaches. Provide discussion for the results.


### For PyTorch-based coding


In [4]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
trainloader = DataLoader(
    trainset, batch_size=batch_size, shuffle=True, num_workers=2, generator=torch.Generator(device)
)

testset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


# Utilities


In [5]:
def train(
    model: torch.nn.Module,
    train_dataloader: DataLoader,
    test_dataloader: DataLoader,
    criterion: torch.nn.Module,
    optimizer: torch.optim.Optimizer,  # type: ignore
    epoch_nums=10,
) -> None:
    train_losses = []
    train_accuracies = []
    test_losses = []
    test_accuracies = []

    for epoch in range(epoch_nums):
        # Train mode
        epoch_loss = torch.tensor(0.0).to(device)
        epoch_accuracy = torch.tensor(0.0).to(device)

        for inputs, labels in train_dataloader:
            inputs: torch.Tensor = inputs.to(device)
            labels: torch.Tensor = labels.to(device)
            outputs: torch.Tensor = model(inputs)

            loss = criterion(outputs, labels)
            epoch_loss += loss

            preds = torch.argmax(outputs, 1)
            epoch_accuracy += (preds == labels).sum()

        print(epoch_loss, epoch_accuracy)
        return

# Exercise 1


## Model from scratch


## Model pre-trained


In [6]:
from torch.optim.rmsprop import RMSprop
from torch.nn import CrossEntropyLoss

pretrained_model = resnet18(weights="IMAGENET1K_V1")
for param in pretrained_model.parameters():
    param.requires_grad = False
pretrained_model.fc = torch.nn.Linear(in_features=512, out_features=len(trainset.targets))
pretrained_model.to(device)

criterion = CrossEntropyLoss().to(device)
optimizer = RMSprop(pretrained_model.fc.parameters(), lr=0.1)
train(pretrained_model, trainloader, testloader, criterion, optimizer, epoch_nums=10)

RuntimeError: Placeholder storage has not been allocated on MPS device!